In [1]:
import os
import sys
import git
import pathlib

import random
import time

import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist
import argparse
import importlib

PROJ_ROOT_PATH = pathlib.Path(git.Repo('.', search_parent_directories=True).working_tree_dir)
PROJ_ROOT =  str(PROJ_ROOT_PATH)
if PROJ_ROOT not in sys.path:
    sys.path.append(PROJ_ROOT)
    
from libs import utils

In [2]:
# set seed
this_seed = 84376
tf.random.set_seed(this_seed)
np.random.seed(this_seed)
random.seed(this_seed)
os.environ['PYTHONHASHSEED']=str(this_seed)

In [3]:
# Tensorflow Limit GPU usage
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)
        

In [4]:
# Extract model
# parser = argparse.ArgumentParser()
# parser.add_argument('model_instance')
# parser.add_argument('error_profile_tag')
# args= parser.parse_args()
model_instance = "fashion-cnn2_1024-1023"   #args.model_instance
error_profile_tag = "LIM_500-2188"          #args.error_profile_tag

In [5]:
print(model_instance, error_profile_tag)

fashion-cnn2_1024-1023 LIM_500-2188


In [6]:
# Prepare dataset
# Combine test and train images together into one dataset
fashion_mnist = tf.keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

train_images = train_images.astype('float32') / 255.0
test_images = test_images.astype('float32') / 255.0

train_images = np.expand_dims(train_images, axis=-1)
test_images = np.expand_dims(test_images, axis=-1)

In [7]:
# Get model
dataset, model_arch, model_config, layer_widths, seed = utils.instancename2metadata(model_instance)
model_meta_type, model_type, model_instance = utils.metadata2instancenames(dataset, model_arch, layer_widths, seed)
model_folder = pathlib.Path(PROJ_ROOT_PATH / "models" / model_type)
model_filename = model_instance + ".h5"
model_file = pathlib.Path(model_folder/ model_filename)

In [8]:
# Load model
model = tf.keras.models.load_model(model_file)

In [9]:
# Load error profile
error_profile_folder = pathlib.Path(PROJ_ROOT_PATH / "error_profiles")
error_profile_filename = error_profile_tag + ".npy"
error_profile_file = pathlib.Path(error_profile_folder/ error_profile_filename)
error_profile = np.load(error_profile_file)
error_lim, error_seed = error_profile_tag.split('-')

In [10]:
# Batchsize for evaluation
EVAL_BATCHSIZE = 128

In [11]:
# Load test set
# Testing with only im_num images
im_num = 128*40#128*40=5120 #[128*78 = 9984]
test_set = (test_images[im_num:], 
            test_labels[im_num:])

In [12]:
error_param = -1 # types of error to optimize for
layer = "c1" # layers to optimize with GA optimization

In [13]:
# meta_optim = avg, ent
# quadrant = q1, q2, q3, q4
# dataset_seg_type = meta_optim +quadrant
dataset_seg_type = "all" # type of dataset segregation

In [14]:
error_type = "ERR_"+str(error_param)
print(error_type)
print("Layer: ", layer)

ERR_-1
Layer:  c1


In [15]:
# RUN Experiment
# Load GA experiment from module depending on layer name
ga_func_name = "EXP_fashion_cnn2_ERR_" + layer
module_name = 'libs.ga.ga_experiments'
module = importlib.import_module(module_name)
ga_func = getattr(module, ga_func_name)
        
# No. of times to repeat experiments
K = 2
POP_SIZE = 3
N_GENERATIONS = 4
CR = 0.6
MR = 0.2

print("Population: ", POP_SIZE)
print("Generations: ", N_GENERATIONS)
print("Crossover Rate: ", CR)
print("Mutation Rate: ", MR)

# Run GA experiment
dataset, model_arch, model_config, layer_widths, seed = utils.instancename2metadata(model_instance)
# model_instance = dataset-modelarch_modelconfig-seed
model_meta_type, model_type, model_instance = utils.metadata2instancenames(dataset, 
                                                                           model_arch, 
                                                                           layer_widths, 
                                                                           seed)

# model_arch = 'cnn'
# model_config = '1024_256_64'
# model_type: 'mnist32-cnn_1024_256_64'
# model_meta_type: 'mnist32-cnn'

EXP_TYPE           = dataset_seg_type + "_"  + model_meta_type + "_" + layer + "_" + error_type
experiment_name    = model_instance   + '--' + error_profile_tag
ga_experiment_name = dataset_seg_type + "_"  + experiment_name + "--" + error_type + '--' + layer + '--ga_' + str(this_seed)
# File/Folder to save log files
logging_folder = pathlib.Path(PROJ_ROOT_PATH / "logging" / dataset_seg_type / model_type / model_instance / error_lim / error_profile_tag / error_type / layer )

if not os.path.exists(logging_folder):
    os.makedirs(logging_folder, exist_ok = True)
logging_filename_tag = pathlib.Path(logging_folder / ga_experiment_name)

for i in range(K):
    print('#' * 20)
    print('Experiment number %d' % (i+1))
    # time_tag = time.strftime("%m%d_%H%M%S")
    logging_filename = str(logging_filename_tag)  + '--' + str(i) + '.csv'
    print("Logfile: ",logging_filename)

    experiment = ga_func(model=model,
                        error_profile=error_profile,
                        ERR_PARAM=error_param,
                        test_set=test_set,
                        log_file=logging_filename,
                        pop_size=POP_SIZE,
                        n_generations=N_GENERATIONS,
                        crossover_rate=CR,
                        mutation_rate=MR,
                        experiment_tag=str(i))

    start = time.perf_counter()
    experiment.run()
    t = time.perf_counter() - start
    print('Experiment number %d: took %f(s)' % (i+1, t))
    print()

Population:  3
Generations:  4
Crossover Rate:  0.6
Mutation Rate:  0.2
####################
Experiment number 1
Logfile:  /repos/faulty_cnn/logging/all/fashion-cnn2_1024/fashion-cnn2_1024-1023/LIM_500/LIM_500-2188/ERR_-1/c1/all_fashion-cnn2_1024-1023--LIM_500-2188--ERR_-1--c1--ga_84376--0.csv
Starting GA experiment...
GA: initializing first population
GA: start main loop...
GA: Generation 1: took 114.53(s)
	Best fitness: 0.789611
GA: Generation 2: took 153.46(s)
	Best fitness: 0.790913
GA: Generation 3: took 77.19(s)
	Best fitness: 0.790913
	No improvement for 1 gen(s)
GA: Generation 4: took 153.30(s)
	Best fitness: 0.792283
Writing result to csv file...
Done
Experiment number 1: took 632.986697(s)

####################
Experiment number 2
Logfile:  /repos/faulty_cnn/logging/all/fashion-cnn2_1024/fashion-cnn2_1024-1023/LIM_500/LIM_500-2188/ERR_-1/c1/all_fashion-cnn2_1024-1023--LIM_500-2188--ERR_-1--c1--ga_84376--1.csv
Starting GA experiment...
GA: initializing first population
GA: sta